<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

# NAML: Neural News Recommendation with Attentive Multi-View Learning
NAML \[1\] is a multi-view news recommendation approach. The core of NAML is a news encoder and a user encoder. The newsencoder is composed of a title encoder, a body encoder, a vert encoder and a subvert encoder. The CNN-based title encoder and body encoder learn title and body representations by capturing words semantic information. After getting news title, body, vert and subvert representations, an attention network is used to aggregate those vectors. In the user encoder, we learn representations of users from their browsed news. Besides, we apply additive attention to learn more informative news and user representations by selecting important words and news.

## Properties of NAML:
- NAML is a multi-view neural news recommendation approach.
- It uses news title, news body, news vert and news subvert to get news repersentations. And it uses user historical behaviors to learn user representations.
- NAML uses additive attention to learn informative news and user representations by selecting important words and news.
- Due to some legal issue, MIND dataset does not release news body. Therefore, we use news abstract instead.

## Data format:
For quicker training and evaluaiton, we sample MINDdemo dataset of 5k users from [MIND small dataset](https://msnews.github.io/). The MINDdemo dataset has the same file format as MINDsmall and MINDlarge. If you want to try experiments on MINDsmall
 and MINDlarge, please change the dowload source.
 Select the MIND_type parameter from ['large', 'small', 'demo'] to choose dataset.
 
**MINDdemo_train** is used for training, and **MINDdemo_dev** is used for evaluation. Training data and evaluation data are composed of a news file and a behaviors file. You can find more detailed data description in [MIND repo](https://github.com/msnews/msnews.github.io/blob/master/assets/doc/introduction.md)

### news data
This file contains news information including newsid, category, subcatgory, news title, news abstarct, news url and entities in news title, entities in news abstarct.
One simple example: <br>

`N46466	lifestyle	lifestyleroyals	The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By	Shop the notebooks, jackets, and more that the royals can't live without.	https://www.msn.com/en-us/lifestyle/lifestyleroyals/the-brands-queen-elizabeth,-prince-charles,-and-prince-philip-swear-by/ss-AAGH0ET?ocid=chopendata	[{"Label": "Prince Philip, Duke of Edinburgh", "Type": "P", "WikidataId": "Q80976", "Confidence": 1.0, "OccurrenceOffsets": [48], "SurfaceForms": ["Prince Philip"]}, {"Label": "Charles, Prince of Wales", "Type": "P", "WikidataId": "Q43274", "Confidence": 1.0, "OccurrenceOffsets": [28], "SurfaceForms": ["Prince Charles"]}, {"Label": "Elizabeth II", "Type": "P", "WikidataId": "Q9682", "Confidence": 0.97, "OccurrenceOffsets": [11], "SurfaceForms": ["Queen Elizabeth"]}]	[]`
<br>

In general, each line in data file represents information of one piece of news: <br>

`[News ID] [Category] [Subcategory] [News Title] [News Abstrct] [News Url] [Entities in News Title] [Entities in News Abstract] ...`

<br>

We generate a word_dict file to tranform words in news title and news abstract to word indexes, and a embedding matrix is initted from pretrained glove embeddings.

### behaviors data
One simple example: <br>
`1	U82271	11/11/2019 3:28:58 PM	N3130 N11621 N12917 N4574 N12140 N9748	N13390-0 N7180-0 N20785-0 N6937-0 N15776-0 N25810-0 N20820-0 N6885-0 N27294-0 N18835-0 N16945-0 N7410-0 N23967-0 N22679-0 N20532-0 N26651-0 N22078-0 N4098-0 N16473-0 N13841-0 N15660-0 N25787-0 N2315-0 N1615-0 N9087-0 N23880-0 N3600-0 N24479-0 N22882-0 N26308-0 N13594-0 N2220-0 N28356-0 N17083-0 N21415-0 N18671-0 N9440-0 N17759-0 N10861-0 N21830-0 N8064-0 N5675-0 N15037-0 N26154-0 N15368-1 N481-0 N3256-0 N20663-0 N23940-0 N7654-0 N10729-0 N7090-0 N23596-0 N15901-0 N16348-0 N13645-0 N8124-0 N20094-0 N27774-0 N23011-0 N14832-0 N15971-0 N27729-0 N2167-0 N11186-0 N18390-0 N21328-0 N10992-0 N20122-0 N1958-0 N2004-0 N26156-0 N17632-0 N26146-0 N17322-0 N18403-0 N17397-0 N18215-0 N14475-0 N9781-0 N17958-0 N3370-0 N1127-0 N15525-0 N12657-0 N10537-0 N18224-0`
<br>

In general, each line in data file represents one instance of an impression. The format is like: <br>

`[Impression ID] [User ID] [Impression Time] [User Click History] [Impression News]`

<br>

User Click History is the user historical clicked news before Impression Time. Impression News is the displayed news in an impression, which format is:<br>

`[News ID 1]-[label1] ... [News ID n]-[labeln]`

<br>
Label represents whether the news is clicked by the user. All information of news in User Click History and Impression News can be found in news data file.

## Global settings and imports

In [1]:
import sys
import os
import numpy as np
from tqdm import tqdm
import papermill as pm
from tempfile import TemporaryDirectory
import tensorflow as tf
import zipfile

sys.path.append("../../")
from reco_utils.recommender.deeprec.deeprec_utils import download_deeprec_resources 
from reco_utils.recommender.newsrec.newsrec_utils import prepare_hparams
from reco_utils.recommender.newsrec.models.naml import NAMLModel
from reco_utils.recommender.newsrec.io.mind_all_iterator import MINDAllIterator
from reco_utils.recommender.newsrec.newsrec_utils import get_mind_data_set


print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))


System version: 3.6.11 | packaged by conda-forge | (default, Aug  5 2020, 20:09:42) 
[GCC 7.5.0]
Tensorflow version: 1.15.2


## Prepare Parameters

In [2]:
epochs=6
seed=42
MIND_type = 'demo'

tmpdir = TemporaryDirectory()

## Download and load data

In [3]:
data_path = tmpdir.name

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding_all.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict_all.pkl")
vertDict_file = os.path.join(data_path, "utils", "vert_dict.pkl")
subvertDict_file = os.path.join(data_path, "utils", "subvert_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'naml.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)
    
if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.blob.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)

100%|██████████| 17.0k/17.0k [00:01<00:00, 9.11kKB/s]
100%|██████████| 9.84k/9.84k [00:01<00:00, 9.52kKB/s]
100%|██████████| 95.0k/95.0k [00:08<00:00, 11.1kKB/s]


## Create hyper-parameters

In [4]:
hparams = prepare_hparams(yaml_file, wordEmb_file=wordEmb_file, \
                          wordDict_file=wordDict_file, userDict_file=userDict_file, \
                          vertDict_file=vertDict_file, subvertDict_file=subvertDict_file, \
                          epochs=epochs)
print(hparams)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

data_format=naml,iterator_type=None,support_quick_scoring=True,wordEmb_file=/tmp/tmpujiq29v7/utils/embedding_all.npy,wordDict_file=/tmp/tmpujiq29v7/utils/word_dict_all.pkl,userDict_file=/tmp/tmpujiq29v7/utils/uid2index.pkl,vertDict_file=/tmp/tmpujiq29v7/utils/vert_dict.pkl,subvertDict_file=/tmp/tmpujiq29v7/utils/subvert_dict.pkl,title_size=30,body_size=50,word_emb_dim=300,word_size=None,user_num=None,vert_num=17,subvert_num=249,his_size=50,npratio=4,dropout=0.2,attention_hidden_dim=200,head_num=4,head_dim=100,cnn_activation=relu,dense_activation=relu,filter_num=400,window_size=3,vert_emb_dim=100,subvert_emb_dim=100,gru_unit=400,

In [5]:
iterator = MINDAllIterator

## Train the NAML model

In [6]:
model = NAMLModel(hparams, iterator, seed=seed)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
print(model.run_eval(valid_news_file, valid_behaviors_file))

18661it [01:21, 229.69it/s]
7475it [00:29, 249.87it/s]
7538it [00:01, 5628.70it/s]


{'group_auc': 0.4814, 'mean_mrr': 0.2103, 'ndcg@5': 0.2143, 'ndcg@10': 0.2771}


In [8]:
model.fit(train_news_file, train_behaviors_file,valid_news_file, valid_behaviors_file)

542it [01:38,  5.51it/s]
18661it [01:19, 234.26it/s]
7475it [00:30, 248.39it/s]
7538it [00:01, 6496.34it/s]
1it [00:00,  5.59it/s]

at epoch 1
train info: logloss loss:1.501282235793082
eval info: group_auc:0.5799, mean_mrr:0.2502, ndcg@10:0.3388, ndcg@5:0.2715
at epoch 1 , train time: 98.3 eval time: 118.3


542it [01:32,  5.86it/s]
18661it [01:19, 234.43it/s]
7475it [00:30, 247.11it/s]
7538it [00:01, 6034.84it/s]
1it [00:00,  5.42it/s]

at epoch 2
train info: logloss loss:1.4345172359934593
eval info: group_auc:0.5838, mean_mrr:0.2561, ndcg@10:0.3437, ndcg@5:0.2758
at epoch 2 , train time: 92.5 eval time: 118.6


542it [01:32,  5.86it/s]
18661it [01:21, 230.10it/s]
7475it [00:30, 247.54it/s]
7538it [00:00, 8813.35it/s]
1it [00:00,  5.30it/s]

...

at epoch 4
train info: logloss loss:1.3584151406569673
eval info: group_auc:0.6363, mean_mrr:0.2894, ndcg@10:0.3831, ndcg@5:0.3198
at epoch 4 , train time: 92.9 eval time: 118.1


542it [01:32,  5.86it/s]
18661it [01:19, 233.42it/s]
7475it [00:29, 250.89it/s]
7538it [00:00, 7800.53it/s]
1it [00:00,  5.59it/s]

at epoch 5
train info: logloss loss:1.3388330485548041
eval info: group_auc:0.6306, mean_mrr:0.287, ndcg@10:0.3812, ndcg@5:0.3152
at epoch 5 , train time: 92.5 eval time: 118.1


542it [01:32,  5.87it/s]
18661it [01:19, 233.56it/s]
7475it [00:30, 247.96it/s]
7538it [00:00, 7664.35it/s]


at epoch 6
train info: logloss loss:1.3178746572719728
eval info: group_auc:0.6415, mean_mrr:0.296, ndcg@10:0.3903, ndcg@5:0.326
at epoch 6 , train time: 92.4 eval time: 118.5


In [9]:
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)
pm.record("res_syn", res_syn)

18661it [01:19, 233.47it/s]
7475it [00:30, 248.54it/s]
7538it [00:01, 7181.57it/s]


{'group_auc': 0.6415, 'mean_mrr': 0.296, 'ndcg@5': 0.326, 'ndcg@10': 0.3903}


/home/miguel/anaconda/envs/reco_gpu/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Function record is deprecated and will be removed in verison 1.0.0 (current version 0.19.1). Please see `scrapbook.glue` (nteract-scrapbook) as a replacement for this functionality.
  This is separate from the ipykernel package so we can avoid doing imports until


## Save the model

In [10]:
model_path = os.path.join(data_path, "model")
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "naml_ckpt"))

## Output Predcition File
This code segment is used to generate the prediction.zip file, which is in the same format in [MIND Competition Submission Tutorial](https://competitions.codalab.org/competitions/24122#learn_the_details-submission-guidelines).

Please change the `MIND_type` parameter to `large` if you want to submit your prediction to [MIND Competition](https://msnews.github.io/competition.html).

In [11]:
group_impr_indexes, group_labels, group_preds = model.run_fast_eval(valid_news_file, valid_behaviors_file)

18661it [01:19, 233.52it/s]
7475it [00:29, 252.92it/s]
7538it [00:01, 6852.74it/s]


In [12]:
with open(os.path.join(data_path, 'prediction.txt'), 'w') as f:
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds)):
        impr_index += 1
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()
        pred_rank = '[' + ','.join([str(i) for i in pred_rank]) + ']'
        f.write(' '.join([str(impr_index), pred_rank])+ '\n')

7538it [00:00, 43079.81it/s]


In [13]:
f = zipfile.ZipFile(os.path.join(data_path, 'prediction.zip'), 'w', zipfile.ZIP_DEFLATED)
f.write(os.path.join(data_path, 'prediction.txt'), arcname='prediction.txt')
f.close()

## Reference
\[1\] Chuhan Wu, Fangzhao Wu, Mingxiao An, Jianqiang Huang, Yongfeng Huang and Xing Xie: Neural News Recommendation with Attentive Multi-View Learning, IJCAI 2019<br>
\[2\] Wu, Fangzhao, et al. "MIND: A Large-scale Dataset for News Recommendation" Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics. https://msnews.github.io/competition.html <br>
\[3\] GloVe: Global Vectors for Word Representation. https://nlp.stanford.edu/projects/glove/